In [13]:
P = 101

In [14]:
def modmul(a, b, m = P):
  return ((a % m) * (b % m)) % m

def mod_pow(a, b, m = P):
  if b == 0:
    return 1
  r = mod_pow(a, b//2, m)
  r = (r*r)%m
  if b%2:
    r = (r*a)%m
  return r

def moddiv(a, b, m = P):
  return modmul(a, mod_pow(b, m-2, m), m)

In [15]:
class Point:
  def __init__(self, x, y):
    self.x = x
    self.y = y
  def __eq__(self, p2):
    return self.x == p2.x and self.y == p2.y
  def __str__(self) -> str:
      return f"({self.x}, {self.y})"

In [16]:
class EllipticCurve:
  def __init__(self, a, b):
    self.a = a
    self.b = b
  
  def add(self, p1, p2, m = P):
    # num - 3p1x^2+a
    # den - 2p1y
    # num - p2y - p1y
    # den - p2x - p1x
    # x3 - (l^2 - p1.x - p2.x) % m
    # y3 = (l*(p1x-x3)-p1y) % m
    l = 0
    if p1 == p2:
      num = 3 * p1.x * p1.x + self.a
      den = 2 * p1.y
    else:
      num = p2.y - p1.y
      den = p2.x - p1.x
    l = moddiv(num, den, m)
    x3 = (l*l - p1.x - p2.x) % m
    y3 = (l*(p1.x - x3) - p1.y) % m
    return Point(x3, y3)

  def mul(self, k, p): # p is always generator point G
    temp = p
    while k != 1:
      temp = self.add(temp, p)
      k -= 1 
    return temp
  
  def sub(self, p1, p2):
    np = Point(p2.x, -p2.y)
    return self.add(p1, np)

In [17]:
# y^2 = x^3 + 2x + 4 
curve = EllipticCurve(2, 4) # Points lying on this curve:{0, 2}, {0, 5}, {1, 0}, {2, 3}, {2, 4}, {3, 3}, {3, 4}, {6, 1}, {6, 6}
G = Point(0, 2) 

In [18]:
def encrypt(p, U):
  # k*G    p + k*U
  k = 5
  c = [
       curve.mul(k, G),
       curve.add(p, curve.mul(k, U))
  ]
  return c

In [19]:
def decrypt(C, R):
  #c1-r*c0
  p = curve.sub(C[1], curve.mul(R, C[0]))
  return p

In [20]:
R = 5               # Private key
U = curve.mul(R, G) # Public key

In [33]:
plaintext = Point(46, 100)

In [34]:
ciphertext = encrypt(plaintext, U)
p = decrypt(ciphertext, R)

In [35]:
print(p)

(46, 100)


In [1]:
def isPrime(n):
    if n==0 or n==1:
        return False
    
    r = int(n/2)
    
    for i in range(2, r):
        if(n%i == 0):
            return False
        
    return True